In [2]:
import pandas as pd

df = pd.read_csv('100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [3]:
# then we want to tokenize the text data
def tokenize(text):
    text = text.lower()
    text = text.replace("?", "")
    text = text.replace("'", "")
    return text.split()

In [5]:
tokenize('what is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [ ]:
# vocab
# hum pure dataset ke sare tokens is vocab me store kregee 
vocab = {'<UNK>': 0}

In [ ]:
# here we are building the vocablurary method
def build_vocab(row):
    # sare qustions and answer ka tokens nikalege row wise 
    tokenized_question = tokenize(row['question'])
    tokenized_answer = tokenize(row['answer'])
    # add kr lege 
    merged_tokens = tokenized_question  + tokenized_answer
    
    # loop chala kar add krege 
    for token in merged_tokens:
        if token not in vocab:
            vocab[token] = len(vocab)

In [8]:
df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [9]:
len(vocab)

324

In [10]:
# convert words to numerical_indices 

def text_to_indices(text, vocab):
    
    indexed_text = []
    
    for token in tokenize(text):
        
        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab['<UNK>'])
            
    return indexed_text            

In [ ]:
# we are using the function to convert text to indices
text_to_indices('what is the capital of Germany?', vocab)

[1, 2, 3, 4, 5, 8]

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class QADataset(Dataset):
    
    def __init__(self, df, vocab):
        self.df = df
        self.vocab = vocab
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        # here we are getting all indices of questions as well as answer 
        numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
        numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)
        
        # atlast we are creating it into the tensor
        return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [15]:
dataset = QADataset(df,vocab)

In [19]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [20]:
for question,answer in dataloader:
    print(question, answer[0])

tensor([[ 10,  75,   3, 296,  19, 297]]) tensor([298])
tensor([[  1,   2,   3, 212,   5,  14, 213, 214]]) tensor([215])
tensor([[ 42, 290, 291, 118, 292, 158, 293, 294]]) tensor([295])
tensor([[ 42, 117, 118,   3, 119,  94, 120]]) tensor([121])
tensor([[  1,   2,   3, 221,   5, 222, 223, 224]]) tensor([225])
tensor([[ 78,  79, 195,  81,  19,   3, 196, 197, 198]]) tensor([199])
tensor([[  1,   2,   3, 146,  86,  19, 192, 193]]) tensor([194])
tensor([[  1,   2,   3,  33,  34,   5, 245]]) tensor([246])
tensor([[ 1,  2,  3, 69,  5, 53]]) tensor([260])
tensor([[ 1,  2,  3, 33, 34,  5, 35]]) tensor([36])
tensor([[ 42,   2,   3, 274, 211, 275]]) tensor([276])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([316])
tensor([[  1,   2,   3,   4,   5, 286]]) tensor([287])
tensor([[ 10,  11, 157, 158, 159]]) tensor([160])
tensor([[  1,   2,   3,   4,   5, 109]]) tensor([317])
tensor([[  1,   2,   3,  37,  38,  39, 161]]) tensor([162])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5, 

In [21]:
import torch.nn as nn

In [30]:
class SimpleRNN(nn.Module):
    
    def __init__(self, vocab_size):
        # Tells PyTorch to track parameters of this model
        super().__init__()
        #  this is converting each_word into the 50 dimensional number 
        # and also giving the size of the vocab for converting it to the vector
        self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
        # then we are applying rnn 
        # in that we are passing the input of 50 and giving the outpu_size of 64 in proper batch wise 
        self.rnn = nn.RNN(50,64 ,batch_first=True)
        # here fc means fully connected 
        # in linear we are connecting all layer and take sentence understanding 64 values ata a time
        self.fc= nn.Linear(64, vocab_size)
        
    def forward(self,question):
            # converting word numbers into vectors 
            embedded_question = self.embedding(question)
            # here we are passing embedded question to rnn
            # where hidden means output for each word 
            # final meand last hidden state
            hidden , final = self.rnn(embedded_question)
            # here we are removing the extra dimension
            output = self.fc(final.squeeze(0))
            
            return output

In [31]:
# vocab size of 324 and each word of 50 dim
x = nn.Embedding(324, embedding_dim=50)
# input size 50, hiddnen size 64
y = nn.RNN(50, 64, batch_first=True)
# takes rnn memory or hidden size of 64 and outputs 324 score 
# one score per word in vocab
z = nn.Linear(64, 324)

# passing first sample of lenght 6 words 
a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
# giving it to the fiest layer as we decide above 
b = x(a)
print("shape of b:", b.shape)
# c is output of every time step
# d is final hidden state
c, d = y(b)
print("shape of c:", c.shape)
# isme output me 1,1,64 aya kyuki hume ek he word ka output milega isme 
print("shape of d:", d.shape)

#  here we are removing one dimension from final hidden state
e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [32]:
learning_rate = 0.001
epochs = 20

In [33]:
model = SimpleRNN(len(vocab))

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [35]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 522.231500
Epoch: 2, Loss: 457.434986
Epoch: 3, Loss: 382.216001
Epoch: 4, Loss: 315.974032
Epoch: 5, Loss: 262.707164
Epoch: 6, Loss: 214.261323
Epoch: 7, Loss: 169.301123
Epoch: 8, Loss: 131.979879
Epoch: 9, Loss: 101.345852
Epoch: 10, Loss: 77.748636
Epoch: 11, Loss: 60.432940
Epoch: 12, Loss: 47.478851
Epoch: 13, Loss: 37.741463
Epoch: 14, Loss: 30.847814
Epoch: 15, Loss: 25.207185
Epoch: 16, Loss: 21.124674
Epoch: 17, Loss: 17.719213
Epoch: 18, Loss: 15.604840
Epoch: 19, Loss: 12.954670
Epoch: 20, Loss: 11.280190


In [36]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [39]:
predict(model, "Who wrote 'To Kill a Mockingbird'?")

harper-lee


In [41]:
list(vocab.keys())[8]

'germany'